In [1]:
import autoassigner
import numpy as np
from gurobipy import GRB, Model

# Helper: Optimal Assignment

First, we provide a naive implementation of the HARD algorithm to compute an optimally fair assignment in exponential time. In several examples below, we will compare the fairness of the PR4A algorithm with the optimal fairness achieved by HARD. 

In [2]:
def hard_assignment(similarity, demand, ability, function = lambda x: x):
    
    problem = Model()
    numrev = similarity.shape[0]
    numpap = similarity.shape[1]
    
    
    #Optimization variables
    frac_assignment = problem.addVars(numrev, numpap, lb=0.0, ub=1.0, vtype=GRB.BINARY) 
    
    
    #Paper- and Reviewer-load constraints
    paperload = problem.addConstrs(frac_assignment.sum('*', paper) == demand for paper in range(numpap))
    revload = problem.addConstrs(frac_assignment.sum(reviewer, '*') <= ability for reviewer in range(numrev))

    #Optimilzation
    
    #Step1. Create an auxiliary variable
    threshold = problem.addVar(vtype=GRB.CONTINUOUS)
    
    #Step2. Request that each paper is assigned reviewers with total similarity higher than threshold
    maximization = problem.addConstrs(sum([frac_assignment[(reviewer, paper)]*function(similarity[reviewer, paper]) 
                                             for reviewer in range(numrev)]) >= threshold 
                                             for paper in range(numpap)) 
    
    #Step3. Maximize threshold threshold
    problem.setObjective(threshold, GRB.MAXIMIZE)

    problem.setParam('OutputFlag', False)
    problem.optimize()
    
    #Construct the assignment

    hard_assignment_paper = {}
    for paper in range(numpap):
        hard_assignment_paper[paper] = [reviewer for reviewer in range(numrev) if 
                                          int(frac_assignment[(reviewer, paper)].X) == 1]
    
    #Output fairness of the assignment and the assignment itself
    return problem.objVal, hard_assignment_paper

## Examples

### Example 1

In this example, we construct a similarity matrix for which the TPMS algorithm outputs an unfair assignment. Indeed, it is not hard to see that in this case TPMS assigns one of the papers to a reviewer who has 0 similarity with this paper. In contrast, PR4A and HARD algorithms assign each paper to a reviewer with a non-zero similarity, both leading the an optimally fair assignment.

In [3]:
similarity0 = np.matrix([[1., 1., 1.], 
                         [0., 0., 1./4-0.01], 
                         [1./4, 1./4, 1./2]])

print(similarity0)

[[1.   1.   1.  ]
 [0.   0.   0.24]
 [0.25 0.25 0.5 ]]


In [4]:
a = autoassigner.auto_assigner(similarity0, 1, 1)
a.fair_assignment()

b = hard_assignment(similarity0, 1, 1)

print('Fairness of the PR4A assignment:', "%.2f" % a.best_quality)
print('Fairness of the HARD assignment:', "%.2f" % b[0])


print('PR4A assignment:', a.fa)

Academic license - for non-commercial use only - expires 2021-07-02
Using license file /Users/stiv/gurobi.lic
Fairness of the PR4A assignment: 0.24
Fairness of the HARD assignment: 0.24
PR4A assignment: {0: [0], 1: [2], 2: [1]}


### Example 2

This example demonstrates how the number of iterations of Steps 2 -- 7 of the PR4A algorithm impacts the assignment. Specifically, we use PR4A to compute two assignments: the first assignment uses only 1 iteration of Steps 2 -- 7 and the second assignment uses 2 iterations.

First, observe that fairness of both assignments is the same (it is always the case -- the first iteration already satisfies fairness guarantees of Theorem 1 in the paper).

Second, observe that two assignments differ in what reviewer is assigned to the second worst-off paper. One-iteration assignment assigns paper 1 to reviewer 0 (similarity 0.04) and paper 2 to reviewer 2 (similarity 1). In contrast, two-iteration assignment assigns paper 1 to reviewer 2 (similarity 0.1) and paper 2 to reviewer 0 (similarity 0.1). Thus, the second iteration helps to promote fairness beyond the most worst-off paper.

In [5]:
similarity1 = np.matrix([[0, 4./100, 10./100],[1./100, 5./100, 0],[0, 10./100, 100./100]])

print(similarity1)

[[0.   0.04 0.1 ]
 [0.01 0.05 0.  ]
 [0.   0.1  1.  ]]


In [6]:
a1_1 = autoassigner.auto_assigner(similarity1, 1, 1, iter_limit=1, time_limit=1)
a1_2 = autoassigner.auto_assigner(similarity1, 1, 1, iter_limit=2, time_limit=1)

a1_1.fair_assignment()
a1_2.fair_assignment()

b1 = hard_assignment(similarity1, 1, 1)



print('Fairness of the PR4A assignment (1 iteration): ', "%.2f" % a1_1.best_quality)
print('Fairness of the PR4A assignment (2 iterations):', "%.2f" % a1_1.best_quality)


print('Fairness of the HARD assignment:', "%.2f" % b1[0])

print('PR4A (1 iteration) assignment: ', a1_1.fa)
print('PR4A (2 iterations) assignment:', a1_2.fa)

Fairness of the PR4A assignment (1 iteration):  0.01
Fairness of the PR4A assignment (2 iterations): 0.01
Fairness of the HARD assignment: 0.01
PR4A (1 iteration) assignment:  {0: [1], 1: [0], 2: [2]}
PR4A (2 iterations) assignment: {0: [1], 1: [2], 2: [0]}


### Example 3

Finally, the last example operates with the similarity matrix from Case C4 of synthetic simulations (Section 8.1 of the paper) for 100 papers and 100 reviewers. Note that in this case we use the MLE transformation function. See the paper for details and discsussion.

In [7]:
lam = 4
k = 2
EPS = 1e-3

numrev = 100
numpap = 100

tmp_similarity2 = np.zeros((k * lam**2, k * lam**2))

tmp = 1./100

tmp_similarity2[0, :k] = 1./lam + EPS
tmp_similarity2[0, k:k*lam] = 1./lam

for itr in range(k):
    tmp_similarity2[itr + 1, itr] = 1
    tmp_similarity2[itr + 1, (k * lam):] = tmp
    
tmp_similarity2[(k+1):, : (k*lam)] = 1.
tmp_similarity2[-k*(lam-1):, (k*lam):] = 1./lam

cur_papers = k*lam**2 - k*lam
cur_row = k*lam**2 - k*lam
cur_col = k*lam

for iteration in range(2, lam + 1):
    if cur_col > k*lam:
        tmp_similarity2[cur_row, cur_col:] = tmp
        cur_row -= 1
        cur_papers -= 1
        cur_col = k*lam
    demand = cur_papers * iteration
    ability = k*lam ** 2
    while (demand - ability) // (lam) > 0:
        tmp_similarity2[cur_row, cur_col:] = tmp
        cur_row -= 1
        demand = demand - iteration 
        cur_col = k*lam
        cur_papers -= 1
    while demand > ability:
        tmp_similarity2[cur_row, cur_col] = tmp
        cur_col += 1
        demand -= 1
    tmp = tmp / 2  

tmp_similarity2 = 10 * tmp_similarity2
tmp_similarity2 = tmp_similarity2 + 1
tmp_similarity2 = tmp_similarity2.T

similarity2 = np.ones((numrev, numpap))
similarity2[0:k * lam**2, 0:k * lam**2] = tmp_similarity2
similarity2[k * lam**2:, k * lam**2:] = 10*np.ones((numrev - k * lam**2, numpap - k * lam**2))

similarity2 = 1 - 1./similarity2

In [8]:
mle_transformation = lambda x: 1 / (1 - x) 

a2 = autoassigner.auto_assigner(similarity2, 4, 4, function=mle_transformation, iter_limit=1, time_limit=1)
a2.fair_assignment()

b2 = hard_assignment(similarity2, 4, 4, function=mle_transformation)

print('Fairness of the PR4A assignment:', "%.2f" % a2.best_quality)
print('Fairness of the HARD assignment:', "%.2f" % b2[0])

Fairness of the PR4A assignment: 6.51
Fairness of the HARD assignment: 14.00
